In [1]:
import os

os.chdir("/home/hamza-arain/Documents/code/HumanActiityTrack")

In [2]:
import pandas as pd
from datetime import datetime
from os import listdir
from os.path import isfile, join
import numpy as np
import joblib

from glob import  glob

In [3]:
df_acceslerometer = pd.read_csv("data/raw_data/A-bench-heavy_MetaWear_2019-01-14T14.22.49.165_C42732BE255C_Accelerometer_12.500Hz_1.4.4.csv")
df_acceslerometer.head(2)

,epoch (ms),time (01:00),elapsed (s),x-axis (g),y-axis (g),z-axis (g)
0,1547472169751,2019-01-14T14:22:49.751,0.00,-0.147,0.702,-0.276
1,1547472169831,2019-01-14T14:22:49.831,0.08,-0.120,0.710,-0.360


In [4]:
df_gyroscope = pd.read_csv("data/raw_data/A-bench-heavy_MetaWear_2019-01-14T14.22.49.165_C42732BE255C_Gyroscope_25.000Hz_1.4.4.csv")
df_gyroscope.head(2)

,epoch (ms),time (01:00),elapsed (s),x-axis (deg/s),y-axis (deg/s),z-axis (deg/s)
0,1547472169353,2019-01-14T14:22:49.353,0.00,-2.866,-0.61,-5.366
1,1547472169393,2019-01-14T14:22:49.393,0.04,-2.500,-0.61,-3.963


In [5]:
file_path = "data/raw_data/"

files = glob(f"{file_path}*.csv")
files[:2]

['data/raw_data/A-squat-medium1-rpe7_MetaWear_2019-01-11T17.05.44.498_C42732BE255C_Accelerometer_12.500Hz_1.4.4.csv',
 'data/raw_data/E-bench-medium_MetaWear_2019-01-18T18.12.13.952_C42732BE255C_Gyroscope_25.000Hz_1.4.4.csv']

In [6]:
print(files[1].split("-"))


participant = files[1].split("-")[0].replace(file_path, "")
label = files[1].split("-")[1]
category = files[1].split("-")[2].rstrip("123") # Remove 1, 2 & 3 from string


['data/raw_data/E', 'bench', 'medium_MetaWear_2019', '01', '18T18.12.13.952_C42732BE255C_Gyroscope_25.000Hz_1.4.4.csv']


In [7]:
df = pd.read_csv(files[1])
df['participant'] = participant
df['label'] = label
df['category'] = category
df['set'] = 1

df

,epoch (ms),time (01:00),elapsed (s),x-axis (deg/s),y-axis (deg/s),z-axis (deg/s),participant,label,category,set
0,1547917934175,2019-01-19 18:12:14.175,0.00,4.451,-6.098,0.244,E,bench,medium_MetaWear_2019,1
1,1547917934215,2019-01-19 18:12:14.215,0.04,3.659,-6.037,0.732,E,bench,medium_MetaWear_2019,1
2,1547917934255,2019-01-19 18:12:14.255,0.08,1.402,-3.902,0.915,E,bench,medium_MetaWear_2019,1
3,1547917934295,2019-01-19 18:12:14.295,0.12,0.305,-2.195,-0.915,E,bench,medium_MetaWear_2019,1
4,1547917934335,2019-01-19 18:12:14.335,0.16,-1.463,-1.524,-0.610,E,bench,medium_MetaWear_2019,1
...,...,...,...,...,...,...,...,...,...,...
553,1547917958295,2019-01-19 18:12:38.295,24.12,-0.549,1.768,1.037,E,bench,medium_MetaWear_2019,1
554,1547917958335,2019-01-19 18:12:38.335,24.16,1.159,-0.732,2.195,E,bench,medium_MetaWear_2019,1
555,1547917958375,2019-01-19 18:12:38.375,24.20,-0.244,-0.610,1.768,E,bench,medium_MetaWear_2019,1
556,1547917958415,2019-01-19 18:12:38.415,24.24,0.244,-1.585,-1.098,E,bench,medium_MetaWear_2019,1


In [8]:
def read_data_from_files(files):
    acc_df = pd.DataFrame()
    gyr_df = pd.DataFrame() 

    acc_set = 1
    gyr_set = 1

    for f in files:
        participant = f.split("-")[0].replace(file_path, "")
        label = f.split("-")[1]
        category = f.split("-")[2].rstrip("123").rstrip("_MetaWear_2019") # Remove 1, 2 & 3 from string

        df = pd.read_csv(f)
        df['participant'] = participant
        df['label'] = label
        df['category'] = category

        if "Accelerometer" in f:
            df['set'] = acc_set
            acc_set += 1
            acc_df = pd.concat([acc_df, df]) 

        if "Gyroscope" in f:
            df['set'] = gyr_set
            gyr_set += 1
            gyr_df = pd.concat([gyr_df, df])

    acc_df.index = pd.to_datetime(acc_df['epoch (ms)'], unit="ms")
    gyr_df.index = pd.to_datetime(gyr_df['epoch (ms)'], unit="ms")

    del acc_df['epoch (ms)']
    del acc_df['time (01:00)']
    del acc_df['elapsed (s)']

    del gyr_df['epoch (ms)']
    del gyr_df['time (01:00)']
    del gyr_df['elapsed (s)']

    return acc_df, gyr_df


In [9]:
files = glob("data/raw_data/*.csv")
acc_df, gyr_df = read_data_from_files(files)

In [10]:
acc_df

,x-axis (g),y-axis (g),z-axis (g),participant,label,category,set
epoch (ms),,,,,,,
2019-01-11 16:05:44.630,0.138,0.721,0.782,A,squat,medium,1
2019-01-11 16:05:44.710,0.144,0.702,0.733,A,squat,medium,1
2019-01-11 16:05:44.790,0.124,0.661,0.691,A,squat,medium,1
2019-01-11 16:05:44.870,0.105,0.640,0.658,A,squat,medium,1
2019-01-11 16:05:44.950,0.090,0.663,0.658,A,squat,medium,1
...,...,...,...,...,...,...,...
2019-01-18 17:26:19.348,0.275,-0.941,0.442,A,rest,standing,94
2019-01-18 17:26:19.428,0.278,-0.939,0.435,A,rest,standing,94
2019-01-18 17:26:19.508,0.280,-0.943,0.433,A,rest,standing,94


In [11]:
gyr_df

,x-axis (deg/s),y-axis (deg/s),z-axis (deg/s),participant,label,category,set
epoch (ms),,,,,,,
2019-01-19 17:12:14.175,4.451,-6.098,0.244,E,bench,medium,1
2019-01-19 17:12:14.215,3.659,-6.037,0.732,E,bench,medium,1
2019-01-19 17:12:14.255,1.402,-3.902,0.915,E,bench,medium,1
2019-01-19 17:12:14.295,0.305,-2.195,-0.915,E,bench,medium,1
2019-01-19 17:12:14.335,-1.463,-1.524,-0.610,E,bench,medium,1
...,...,...,...,...,...,...,...
2019-01-20 17:33:27.647,3.537,-5.061,2.683,E,row,medium,93
2019-01-20 17:33:27.687,-0.305,-1.829,1.402,E,row,medium,93
2019-01-20 17:33:27.727,-0.488,3.659,-2.195,E,row,medium,93


In [12]:
merged_df = pd.concat([acc_df.iloc[:, :3], gyr_df], axis=1)
merged_df.columns = [
    "acc_x",
    "acc_y",
    "acc_z",
    "gyr_x",
    "gyr_y",
    "gyr_z",
    "participant", 
    "label",
    "category",
    "set"
]
merged_df

,acc_x,acc_y,acc_z,gyr_x,gyr_y,gyr_z,participant,label,category,set
epoch (ms),,,,,,,,,,
2019-01-11 15:08:04.950,NaN,NaN,NaN,-10.671,-1.524,5.976,B,bench,heavy,53.0
2019-01-11 15:08:04.990,NaN,NaN,NaN,-8.720,-2.073,3.171,B,bench,heavy,53.0
2019-01-11 15:08:05.030,NaN,NaN,NaN,0.488,-3.537,-4.146,B,bench,heavy,53.0
2019-01-11 15:08:05.070,NaN,NaN,NaN,0.244,-5.854,3.537,B,bench,heavy,53.0
2019-01-11 15:08:05.110,NaN,NaN,NaN,-0.915,0.061,-2.805,B,bench,heavy,53.0
...,...,...,...,...,...,...,...,...,...,...
2019-01-20 17:35:13.382,-0.060,-1.021,-0.058,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-20 17:35:13.462,-0.035,-1.037,-0.026,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-20 17:35:13.542,-0.045,-1.029,-0.033,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
sampling = {
    "acc_x": "mean",
    "acc_y": "mean",
    "acc_z": "mean",
    "gyr_x": "mean",
    "gyr_y": "mean",
    "gyr_z": "mean",
    "participant": "last", 
    "label": "last",
    "category": "last",
    "set": "last"
}

merged_df[:1000].resample(rule="200ms").apply(sampling)

,acc_x,acc_y,acc_z,gyr_x,gyr_y,gyr_z,participant,label,category,set
epoch (ms),,,,,,,,,,
2019-01-11 15:08:04.800,NaN,NaN,NaN,-9.69550,-1.7985,4.5735,B,bench,heavy,53.0
2019-01-11 15:08:05.000,NaN,NaN,NaN,-2.01220,-0.4270,-1.6218,B,bench,heavy,53.0
2019-01-11 15:08:05.200,0.013500,0.977000,-0.071000,-1.89040,2.4392,0.9388,B,bench,heavy,53.0
2019-01-11 15:08:05.400,-0.001500,0.970500,-0.079500,-1.68260,-0.8904,2.1708,B,bench,heavy,53.0
2019-01-11 15:08:05.600,0.001333,0.971667,-0.064333,2.56080,-0.2560,-1.4146,B,bench,heavy,53.0
...,...,...,...,...,...,...,...,...,...,...
2019-01-11 15:10:21.600,-0.174000,1.092667,-0.153333,-4.96360,-16.4026,21.9026,A,bench,heavy,85.0
2019-01-11 15:10:21.800,-0.219500,1.402500,-0.109000,-7.52440,18.6706,-22.6828,A,bench,heavy,85.0
2019-01-11 15:10:22.000,-0.192000,0.871667,-0.155667,2.60980,8.7070,-14.8658,A,bench,heavy,85.0


In [14]:
days = [g for n, g in merged_df.groupby(pd.Grouper(freq="D"))]
days

[                         acc_x  acc_y  acc_z   gyr_x   gyr_y  gyr_z  \
 epoch (ms)                                                            
 2019-01-11 15:08:04.950    NaN    NaN    NaN -10.671  -1.524  5.976   
 2019-01-11 15:08:04.990    NaN    NaN    NaN  -8.720  -2.073  3.171   
 2019-01-11 15:08:05.030    NaN    NaN    NaN   0.488  -3.537 -4.146   
 2019-01-11 15:08:05.070    NaN    NaN    NaN   0.244  -5.854  3.537   
 2019-01-11 15:08:05.110    NaN    NaN    NaN  -0.915   0.061 -2.805   
 ...                        ...    ...    ...     ...     ...    ...   
 2019-01-11 16:24:55.706    NaN    NaN    NaN  -8.415   7.683  4.207   
 2019-01-11 16:24:55.746    NaN    NaN    NaN  -4.573  -5.732  2.744   
 2019-01-11 16:24:55.753  0.005 -1.022 -0.176     NaN     NaN    NaN   
 2019-01-11 16:24:55.786    NaN    NaN    NaN   1.463 -12.195 -0.305   
 2019-01-11 16:24:55.833 -0.002 -1.026 -0.189     NaN     NaN    NaN   
 
                         participant  label category   set  
 

In [15]:
resmapled_df = pd.concat([df.resample(rule="200ms").apply(sampling).dropna() for df in days])
resmapled_df

,acc_x,acc_y,acc_z,gyr_x,gyr_y,gyr_z,participant,label,category,set
epoch (ms),,,,,,,,,,
2019-01-11 15:08:05.200,0.013500,0.977000,-0.071000,-1.8904,2.4392,0.9388,B,bench,heavy,53.0
2019-01-11 15:08:05.400,-0.001500,0.970500,-0.079500,-1.6826,-0.8904,2.1708,B,bench,heavy,53.0
2019-01-11 15:08:05.600,0.001333,0.971667,-0.064333,2.5608,-0.2560,-1.4146,B,bench,heavy,53.0
2019-01-11 15:08:05.800,-0.024000,0.957000,-0.073500,8.0610,-4.5244,-2.0730,B,bench,heavy,53.0
2019-01-11 15:08:06.000,-0.028000,0.957667,-0.115000,2.4390,-1.5486,-3.6098,B,bench,heavy,53.0
...,...,...,...,...,...,...,...,...,...,...
2019-01-20 17:33:27.000,-0.048000,-1.041500,-0.076500,1.4146,-5.6218,0.2926,E,row,medium,93.0
2019-01-20 17:33:27.200,-0.037000,-1.030333,-0.053333,-2.7684,-0.5854,2.2440,E,row,medium,93.0
2019-01-20 17:33:27.400,-0.060000,-1.031000,-0.082000,2.8416,-5.1342,-0.1220,E,row,medium,93.0


In [16]:
resmapled_df['set'] = resmapled_df['set'].astype("int")

resmapled_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 9009 entries, 2019-01-11 15:08:05.200000 to 2019-01-20 17:33:27.800000
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   acc_x        9009 non-null   float64
 1   acc_y        9009 non-null   float64
 2   acc_z        9009 non-null   float64
 3   gyr_x        9009 non-null   float64
 4   gyr_y        9009 non-null   float64
 5   gyr_z        9009 non-null   float64
 6   participant  9009 non-null   object 
 7   label        9009 non-null   object 
 8   category     9009 non-null   object 
 9   set          9009 non-null   int64  
dtypes: float64(6), int64(1), object(3)
memory usage: 774.2+ KB


In [17]:
import joblib

joblib.dump(resmapled_df, "data/01_processed_data.joblib")

['data/01_processed_data.joblib']

## Final Code

In [18]:
def process_data(files):
    acc_df = pd.DataFrame()
    gyr_df = pd.DataFrame() 

    acc_set = 1
    gyr_set = 1

    for f in files:
        participant = f.split("-")[0].replace(file_path, "")
        label = f.split("-")[1]
        category = f.split("-")[2].rstrip("123").rstrip("_MetaWear_2019") # Remove 1, 2 & 3 from string

        df = pd.read_csv(f)
        df['participant'] = participant
        df['label'] = label
        df['category'] = category

        if "Accelerometer" in f:
            df['set'] = acc_set
            acc_set += 1
            acc_df = pd.concat([acc_df, df]) 

        if "Gyroscope" in f:
            df['set'] = gyr_set
            gyr_set += 1
            gyr_df = pd.concat([gyr_df, df])

    acc_df.index = pd.to_datetime(acc_df['epoch (ms)'], unit="ms")
    gyr_df.index = pd.to_datetime(gyr_df['epoch (ms)'], unit="ms")

    del acc_df['epoch (ms)']
    del acc_df['time (01:00)']
    del acc_df['elapsed (s)']

    del gyr_df['epoch (ms)']
    del gyr_df['time (01:00)']
    del gyr_df['elapsed (s)']

    # return acc_df, gyr_df

    merged_df = pd.concat([acc_df.iloc[:, :3], gyr_df], axis=1)
    merged_df.columns = [
        "acc_x",
        "acc_y",
        "acc_z",
        "gyr_x",
        "gyr_y",
        "gyr_z",
        "participant", 
        "label",
        "category",
        "set"
    ]

    sampling = {
        "acc_x": "mean",
        "acc_y": "mean",
        "acc_z": "mean",
        "gyr_x": "mean",
        "gyr_y": "mean",
        "gyr_z": "mean",
        "participant": "last", 
        "label": "last",
        "category": "last",
        "set": "last"
    }

    days = [g for n, g in merged_df.groupby(pd.Grouper(freq="D"))]
    resmapled_df = pd.concat([df.resample(rule="200ms").apply(sampling).dropna() for df in days])

    joblib.dump(resmapled_df, "data/01_processed_data.joblib")
    return resmapled_df

file_path = "data/raw_data/"

files = glob(f"{file_path}*.csv")
df = process_data(files)

In [19]:
df

,acc_x,acc_y,acc_z,gyr_x,gyr_y,gyr_z,participant,label,category,set
epoch (ms),,,,,,,,,,
2019-01-11 15:08:05.200,0.013500,0.977000,-0.071000,-1.8904,2.4392,0.9388,B,bench,heavy,53.0
2019-01-11 15:08:05.400,-0.001500,0.970500,-0.079500,-1.6826,-0.8904,2.1708,B,bench,heavy,53.0
2019-01-11 15:08:05.600,0.001333,0.971667,-0.064333,2.5608,-0.2560,-1.4146,B,bench,heavy,53.0
2019-01-11 15:08:05.800,-0.024000,0.957000,-0.073500,8.0610,-4.5244,-2.0730,B,bench,heavy,53.0
2019-01-11 15:08:06.000,-0.028000,0.957667,-0.115000,2.4390,-1.5486,-3.6098,B,bench,heavy,53.0
...,...,...,...,...,...,...,...,...,...,...
2019-01-20 17:33:27.000,-0.048000,-1.041500,-0.076500,1.4146,-5.6218,0.2926,E,row,medium,93.0
2019-01-20 17:33:27.200,-0.037000,-1.030333,-0.053333,-2.7684,-0.5854,2.2440,E,row,medium,93.0
2019-01-20 17:33:27.400,-0.060000,-1.031000,-0.082000,2.8416,-5.1342,-0.1220,E,row,medium,93.0
